### imports

In [1]:
import numpy as np
import strange
import toytree
from scipy import stats
import ipyparallel as ipp
import h5py
from collections import Counter
from copy import deepcopy
from numba import jit
import pandas as pd

### make tree

In [2]:
rtr = toytree.rtree().coaltree(ntips=8,seed=42)
rtr3 = rtr.mod.node_scale_root_height(3)
rtr3.draw();

r7 r2 r1 r4 r3 r0 r5 r6

### simulate!

In [3]:
Ne = 50000
mut = 1e-8
kwargs = {
    "workdir": "../tests",
    "mutation_rate": mut,
    "recombination_rate": 1e-9,
    "theta": Ne*mut*4,
    "length": int(1e6), 
    "get_sequences": True,
    "random_seed": 42,
}

# simulation object
coal8 = strange.Coalseq(tree=rtr3, name="coal8", **kwargs)

In [4]:
coal8.tree_table.head(10)

,end,length,mstree,nsnps,start,treeheight
0,54,54,"((8:204293.32434399396880,(6:64174.00872287491...",2,0,473009
1,568,514,"((8:204293.32434399396880,(6:64174.00872287491...",12,54,473009
2,675,107,"((8:204293.32434399396880,(6:64174.00872287491...",4,568,473009
3,2454,1779,"((8:204293.32434399396880,(6:64174.00872287491...",29,675,473009
4,2515,61,"((8:204293.32434399396880,(6:64174.00872287491...",4,2454,473009
5,2655,140,"((8:204293.32434399396880,(6:64174.00872287491...",6,2515,546145
6,2960,305,"((8:204293.32434399396880,(6:64174.00872287491...",5,2655,546145
7,3420,460,"((8:204293.32434399396880,(6:64174.00872287491...",13,2960,546145
8,3690,270,"(((4:104748.95632937064511,5:104748.9563293706...",6,3420,546145
9,4288,598,"(((4:104748.95632937064511,5:104748.9563293706...",9,3690,546145


### Let's run MrBayes on each gene tree

In [2]:
ipyclient = ipp.Client()
ipyclient

In [3]:
sliding_obj = strange.SlidingWindow(name='coal8',workdir='../tests/',ipyclient=ipyclient)

In [4]:
sliding_obj.add_probs_topokey()

[####################] 100% 0:03:41 | computing gene tree probabilities 

In [8]:
sliding_obj.run_mb_mstrees()

[####################] 100% 0:14:36 | inferring mb trees on mstrees 
consolidating...


### So now we've run MrBayes on each gene tree, made a key that has indexed every observed topology, and produced a table of how often each topology was visited during each gene tree mcmc. 

We might want to also compute the probability, for each visited topology, of that topology being produced under the species tree. We can do that!

In [5]:
sliding_obj.add_probs_topokey()

[                    ]   3% 0:00:08 | computing gene tree probabilities 

KeyboardInterrupt: 

### Now let's do some resampling.

make an mcmc object:

In [2]:
mcmcobj= strange.MBmcmc('../tests/coal8_mb_mstrees_mcmc.csv')

### each column corresponds to an msprime gene segment, and each row corresponds to a topology visited by the mcmc.

In [3]:
mcmcobj.mbcsv

array([[6144, 2811, 4627, ...,  542,  421, 1828],
       [6144, 2811, 4627, ...,  542,  421, 1828],
       [6144, 2811, 4627, ...,  542,  421, 1828],
       ...,
       [1655, 6357, 8622, ..., 2323, 8826, 4284],
       [1655, 6357, 8622, ..., 2323, 8826, 4284],
       [1655, 6357, 8622, ..., 2323, 8826, 4284]])

### Now, we can resample a bunch of times:

In [4]:
mcmcobj.update_x_times(arr=mcmcobj.mbcsv,
                      mixnum=5,
                      num_times = 500,
                      sd_normal = 1)

### after 500 resamples...

In [5]:
mcmcobj.mbcsv

array([[5319, 2811, 4627, ...,  542,  421, 8956],
       [6144, 2811, 1346, ...,  542,  421, 1828],
       [5392, 2811, 4627, ...,  542,  421, 1828],
       ...,
       [1655, 4473, 9107, ..., 2323, 8826, 7569],
       [1655, 6357, 8622, ..., 2323,  421, 4284],
       [5485, 6357, 8622, ..., 2323,  421, 4284]])

### ...you can see that it looks a bit different.

#  

### What's the most frequent topology for each segment after sampling?

In [7]:
strange.Window.mode(mcmcobj.mbcsv)[0]

array([2964, 4473, 4627, ...,  542,  421, 2656])

In [8]:
from collections import Counter

Counter({11: 1,
         21: 1,
         37: 1,
         43: 4,
         73: 1,
         112: 1,
         119: 1,
         133: 1,
         134: 1,
         139: 1,
         173: 1,
         257: 3,
         278: 1,
         284: 1,
         294: 2,
         322: 1,
         346: 4,
         355: 1,
         384: 1,
         411: 1,
         416: 1,
         421: 12,
         459: 2,
         464: 2,
         495: 1,
         511: 2,
         536: 1,
         542: 3,
         590: 1,
         595: 1,
         601: 2,
         613: 7,
         617: 1,
         628: 2,
         645: 1,
         648: 1,
         651: 1,
         662: 1,
         663: 1,
         670: 1,
         686: 1,
         693: 1,
         695: 2,
         697: 1,
         707: 1,
         712: 2,
         724: 1,
         745: 1,
         746: 1,
         786: 1,
         788: 2,
         843: 1,
         866: 1,
         870: 1,
         874: 1,
         920: 2,
         934: 2,
         959: 1,
         967: 1,
 